# Transactions

## Transaction
### Definition
A logical unit of work that must either be entirely completed or aborted (indivisible, atomic), a sequence of DML statements

DML statements are already atomic; DBMS also allows for _user-defined_ transactions

A successful transaction changes the database from one consistent state to another

### Properties (ACID)
1. **Atomicity**<br/>
A transaction is a single, indivisible, logical unit of work. All operations in a transaction must be completed, or the transaction is aborted.
2. **Consistency**<br/>
Constraints that hold before a transaction must also hold after it<br/>
Multiple users accessing the same data see the same value
3. **Isolation**<br/>
Changes made during execution of a transaction cannot be seen by other transactions until this one is completed
4. **Durability**<br/>
When a transaction is complete, the changes made to the database are permanent, even if the system fails

### Why do we need a transaction?
Brief: 1. For single users: makes sure that data is consistent (if crashes); 2. For multi-users: prevents concurrency-related conflicts in data reading + writing 

#### Problem 1: Unit of work
Users need the ability to define a unit of work

multiple statements (user-defined transaction)

<center>
 <img src="./figures/week9-1.png" width = "200" alt="图片名称" align=center />
</center>

SQL keywords: `begin/START TRANSACTION`, `commit`, `rollback`.

In the case of an error:
1. Any SQL statements already completed must be reversed
2. Show an error message to the user
3. When ready, the user can try the transaction again
This is briefly annoying, but inconsistent data is disastrous.

#### Problem 2: Concurrent access
Concurrent access to data by > 1 user or program

Concurrent execution of DML against a shared database.

Problems:
1. Lost updates (no updates)
<center>
 <img src="./figures/week9l2-2.png" width = "300" alt="图片名称" align=center />
</center>

2. Uncommitted data<br/>
When two transactions execute concurrently and the first is rolled back after the second has already accessed the uncommitted data.
<center>
 <img src="./figures/week9l2-3.png" width = "300" alt="图片名称" align=center />
</center>

3. Inconsistent retrievals<br/>
When one transaction calculates some aggregate functions over a set of data, while other transactions are updating the data (some data are read after change and some before, inconsistently)<br/>
<center>
 <img src="./figures/week9l2-4.png" width = "300" alt="图片名称" align=center />
</center>

### Serializability
Transactions ideally are serializable.

Multiple, concurrent transactions appear as if they were executed one after another

Ensures that the concurrent execution of several transactions yields consistent results.

But true setial execution (no concurrency) is very expensive.

### Logging 
Allow us to restore the database to a previous consistent state (e.g. not completed, aborted => roll back; restore a corrupted database)

By tracking all updates to data. Contains:
1. A record for the beginning of the transaction
2. For each SQL statement: operation being performed (e.g. update, delete, insert); objects affected by the transaction; before and after values for updated fields; pointers to previous and next transaction log entries
3. COMMIT (ending of the transaction)

When failure occurs, DBMS will examine the log for all uncommitted or incomplete transactions, and restore the database to a precious state.

## Concurrency 
TO achieve efficient execution of transactions, the DBMS creates a schedule of read and write operations for concurrent transactions.
Interleaves the execution of operations, based on concurrency control algorithms:
1. Locking (Main method)
2. Time stamping
3. Optimistic Concurrency Control 

### Control: Locking
#### Basics
**Lock:** Guarantees exclusive use of a data item to a current transaction
 Required to prevent another transaction from reading inconsistent data

**Lock manager:** Responsible for assigning and policing the locks used by the transactions

#### Lock Granularity Options
1. Database-level lock<br/>
Entire database is locked; T1 and T2 can not access the same data base concurrently even if they use different tables<br/>
Good for batch processing but unsuitable for multi-user DBMSs
2. Table-level lock<br/>
Entire table is locked; T1 and T2 can access the same database concurrently as long as they use different tables<br/>
Can cause bottlenecks: transactions want to access different parts of the table; Not suitable for highly multi-user DBMSs
3. Page-level lock<br/>
An entire disk page is locked; <br/>
Not commonly used now
4. Row-level lock<br/>
Allows concurrent transactions to access different rows of the same table, even if the rows are located on the same page<br/>
Improves data availability but with high overhead (each row has a lock that must be read and written to)<br/>
Currently the most popular approach (MySQL, Oracle)
5. Field-level lock<br/>
Allows concurrent transactions to access the same row, as long as they access different attributes within that row<br/>
Most flexible lock but requires an extremely high level of overhead<br/>
__Not commonly used__

Types of Locks: Binary Lock & Shared and Exclusive (/Read and Write) Locks

#### Type: Binary Locks
 1(locked)/0(unlocked); the lock is not released until the statement is completed

Eliminates "Lost Update" problem

But too restrictive to yield optimal concurrency, as even two READS are locked

#### Type: Shared and Exclusive Locks
- Exclusive Lock: (when transaction intends to WRITE)<br/>
Access is reserved for the transaction that locked the object; Granted iff no other locks are held on the data item (both exclusive & shared locks)<br/>
e.g. MySQL: `SELECT ... FOR update`
- Shared Lock: (when a transaction wants to READ)<br/>
Other transactions are also granted Read access; Issued when no Exclusive lock is held on that data item (can have multiple shared locks)<br/>
e.g. MySQL: `SELECT ... FOR share`

#### Deadlock
When two transactions wait for each other to unlock data with exclusive locks (could wait forever)

**Solutions**
1. Prevention
2. Detection (then kill one of them; e.g. MySQL-kill the 2nd one)


### Control: Alternative concurrency control methods
1. Timestamp
- Assigns a global unique timestamp to each transaction
- Each data item accessed by the transaction gets the timestamp
- When a transaction wants to read or write, the DBMS compares its timestamp with the timestamps already attached to the item, decides whether to allow access

2. Optimistic
- Based on the **assumption** that the majority of database operations do not conflict
- Transaction is executed without restrictinos or checking
- When commit, the DBMS checks whether any of the data it read has been altered, if so then rollback



<center>
 <img src="./figures/week9l1-2.png" width = "350" alt="图片名称" align=center />
</center>







# Database Administration
The role of the DBA: Capacity planning and Backup and recovery

## Capacity planning
"is the process of predicting when future load levels will saturate the system and determining the most cost-effective way of delaying system saturation as much as possible."

Consider: disk space requirements; transaction throughput; go-live and throughout the life of the system storage (7~20 years)

Consider: Data volumes; Access Frequencies

### Estimating disk space requirement
**Core Idea**: Treat database size as the sum of all table sizes
$$Table\ size=number\ of\ rows \times average\ row\ width$$

<center>
 <img src="./figures/week9l2-6.png" width = "250" alt="图片名称" align=center />
</center>

1. Calculating row widths (VARCHAR/BLOB use the average size from catalog)
2. Estimate growth of tables: Gather estimates during system analysis;<br/>
**Event Tables**-most frequent tables; dominant the storage later

<center>
 <img src="./figures/week9l2-7.png" width = "300" alt="图片名称" align=center />
</center>
<br/>
<center>
 <img src="./figures/week9l2-8.png" width = "500" alt="图片名称" align=center />
</center>

### Estimate Transaction Load
1. how often will transaction each be run? 
2. for each transaction, what SQL statements are being run?

<center>
 <img src="./figures/week9l2-9.png" width = "350" alt="图片名称" align=center />
</center>


### Summary
In fact, also need to store index; so much more!!

remember the size at go-live; how fast it grows; Frequencies of accessing

NOTE: capacity planning is a rough estimation. (but good enough)



## Backup and Recovery
### Why? Protect data from
1. **Human error**: e.g. accidental drop or delete
2. **Hardware or software malfunction**: e.g. bug in application, hard drive, CPU, memory
3. **Malicious activity**: e.g. security compromise (server, database, application)
4. **Natural or man made disasters**: need consider the scale of the damage
5. **Government Regulation**: e.g. historical archiving rules, Metadata collection, HIPPA,EU data retention regulations, Privacy Rules

#### Categories of Failure
1. Statement failure: Syntactically incorrect
2. User Process failure: The process doing the work fails (errors, dies)
3. Network failure: between the user and the database
4. User error: User accidentally drop the row, table, database
5. Memory failure: Memory fails, becomes corrupt
6. Media failure: Disk failure, corruption, deletion

### Backup Types
#### Physical vs Logical
- **Physical** (Binary data being stored, clone the HDD)<br/>
backup = exact copies of the database directories and files<br/>
Database is preferably offline (cold backup) when backup occurs (MySQL is not wholly offline)<br/>
Backup is only portable to machines with a similar configuration<br/>
Suiltable for large databases that need fast recovery<br/>
**Restore:** <br/>
    1.shut down DBMS<br/>
    2.copy backup over current structure on disk<br/>
    3.restart DBMS

- **Logical** (export of data as SQL statements)<br/>
backup completed through SQL queries; doesn't include log/config files<br/>
Server is available during the backup; machine independent<br/>
Slower than physical; output is larger than physical<br/>
in MySQL: `Mysqldump`/`SELECT...INTO OUTFILE`<br/>
**Restore:** `mysqlimport`/`LOAD DATA INFILE`

#### Online vs Offline
- **Online (or Hot)**<br/>
backups occur when the database is live<br/>
Need to have appropriate locking to ensure integrity of data<br/>
clients don't realise a backup is in progress

- **Offline (or COLD)**<br/>
backups occur when the database is stopped<br/>
Take backup from replication server not live server (To maximize availability to users)<br/>
Simpler to perform; preferable, but not available in all situations (e.g. applications without downtime)

#### Full vs Incremental
- **FUll**<br/>
The complete database is backed up (physical/logical/online/offline)<br/>
Includes everything you need to get the database operational in the event of a failure

- **Incremental**<br/>
Only the changes since the last backup are backuped; most databases: only backup log files<br/>
**Restore**:<br/>
    1.Stop the database, copy backed up log files to disk<br/>
    2.start the database and tell it to redo the log files

<center>
 <img src="./figures/week9l2-10.png" width = "400" alt="图片名称" align=center />
</center>

#### Onsite vs Offsite
- **Onsite**: the same equipment you are using now / Keep at same site as server (but different computer)
- **offsite**: store it somewhere else / Keep everything at a physically removed site (>160km = 100Mi)<br/>
Enables disaster recovery<br/>
Examples: backup tapes transported to underground vault; remote mirror database maintained via replication; backup to Cloud

<center>
 <img src="./figures/week9l2-5.png" width = "400" alt="图片名称" align=center />
</center>

### Backup Policy
Backup strategy is usually a combination of full and incremental backups. (e.g. weekly full backup, weekday incremental backup)

Conduct backups when database load is low

If using replication, use the mirror database for backups to negate any performance concerns with the primary database

**CRUTIAL: TEST your backup before you NEED your backup**
